### scrapper

In [1]:
import requests
from PIL import Image
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from io import BytesIO
import pandas as pd
from openpyxl import load_workbook

from selenium.webdriver import Chrome
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time


#from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions
#from azure_key import account_name, account_key, container_name

import psycopg2
#from sql_key import dbname, user, password, host, port


In [2]:
from DB_and_Azure import sql_db_functions as SQLf, azure_functions as Af

In [3]:
def create_soup(url):

    chrome_options = Options()
    #chrome_options.add_argument("--headless")  # Ensure GUI is off
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")


    browser = Chrome(options=chrome_options)

    
    # Open the URL
    browser.maximize_window()
    browser.get(url)
    
    # Wait for the page to fully load
    wait = WebDriverWait(browser, 120)
    wait.until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
    
    # Wait for the cookie button to be clickable
    cookie_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.Button_button__qoiWI.CookiesContent_button__2Vj_R')))
    cookie_button.click()

    # Scroll to the bottom of the page
    last_height = browser.execute_script("return document.body.scrollHeight")
    
    while True:
        for i in range(5):  # 5 iterations to make 20% scroll each
            browser.execute_script("window.scrollBy(0, document.body.scrollHeight / 5);")
            time.sleep(1)  # Wait for 1 second between each scroll
        
        # Calculate new scroll height and compare with the last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


    # Get the page source after scrolling
    html = browser.page_source

    
 
    browser.quit()
     


    page_soup = BeautifulSoup(html, 'html.parser')

    return page_soup


In [4]:
def create_simple_soup(prod_link):
    ua = UserAgent()
    header = {'User-Agent':str(ua.chrome)}

    # Send an HTTP request to the URL
    response = requests.get(prod_link, headers = header)
    prod_soup = BeautifulSoup(response.content, 'html.parser')

    return prod_soup

In [9]:
def contains_required_prods(line):
    
    keeping = ["Articolo senza modello","Dettaglio dell'articolo 8","Retro dell'articolo"]


    return any(num in line for num in keeping)

In [10]:
def get_image_links(prod_soup):
    
    item_list_1 = prod_soup.select('ul li button img')

    ##### get image
    images = []
    i = 0
    for item in item_list_1: 
            
        if contains_required_prods(item['alt']) :
            images = images + [item['src']]
        
        i += 1


    
    print(images)
    return images

In [11]:
def get_description(prod_soup):
    
    description = prod_soup.select('div.Description_description__IDgi6 p')[0].text
    description = description + ' ' + prod_soup.select('div.ProductDetail_properties__UStvB div.PropertiesContent_content__T7Hg5 p')[0].text
    

    return description

    

In [12]:
def links_to_images(links,prod_name,container_client):

    ua = UserAgent()
    header = {'User-Agent':str(ua.chrome)}

    i = 0
    for img_link in links:
        response = requests.get(img_link, headers = header)
        img = Image.open(BytesIO(response.content))

        # Resize the image to 600
        img_resized = img.resize((500, 500))

        img_name = f"Gucci{prod_name}_{i}.jpg"

        # Load to Blob
        #Af.images_to_blob(img, img_name,container_client)
        print(img_name)
        i+=1



In [28]:
import re

def get_price(prod_soup):

    text = prod_soup.select('span.SinglePrice_end__Hz2J7')[0].text


    # Remove any non-numeric characters except for ',' and '.'
    cleaned_text = re.sub(r'[^\d,\.]', '', text)
    
    # Replace comma with a period if there's no period already (to handle decimal part)
    if ',' in cleaned_text and '.' not in cleaned_text:
        cleaned_text = cleaned_text.replace(',', '.')
    elif ',' in cleaned_text and '.' in cleaned_text:
        # If both ',' and '.' are present, keep only the period as the decimal separator
        cleaned_text = cleaned_text.replace(',', '')
    
    # Convert the string to a float
    number = float(cleaned_text)
    
    return number

In [12]:
def mango(category_url,n_products):

    soup = create_soup(category_url)
    item_list = soup.select('div.ProductImage_productImage__cS5d9')

    
    #container_client = Af.connect_blob()
    container_client = True

    #conn, cursor = SQLf.connect_sql()
    conn = True
    cursor = True


    br = 0
    for item in item_list: 
            
        if br == n_products:
            break
        else:

            #Scrapp
            prod_soup = create_simple_soup(item.a['href'])

            prod_images_links = get_image_links(prod_soup=prod_soup)
            prod_description = get_description(prod_soup=prod_soup)
            prod_price = get_price(prod_soup=prod_soup)

            prod_name = 'Mango_'+str(br)  
            
            # Links to image, load to blob and return prod_images_names
            prod_images_names = links_to_images(links = prod_images_links,prod_name= br,container_client=container_client)
            
            SQLf.data_to_SQL(conn= conn ,prod_name=prod_name, prod_description=prod_description,brand='HM',prod_images_names=prod_images_names)


        br += 1


# Execute scrapper 

In [13]:
link = 'https://shop.mango.com/it/it/c/donna/taglie-plus/giacchette_5fd4b931'

mango(link,3)

['https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67037113_43_R.jpg?imwidth=2048&imdensity=1&ts=1711469518224', 'https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67037113_43_D8.jpg?imwidth=2048&imdensity=1&ts=1701765195306', 'https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67037113_43_B.jpg?imwidth=2048&imdensity=1&ts=1701765195306']
Gucci0_0.jpg
Gucci0_1.jpg
Gucci0_2.jpg

['https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67041009_99_R.jpg?imwidth=2048&imdensity=1&ts=1710108420002', 'https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67041009_99_D8.jpg?imwidth=2048&imdensity=1&ts=1697727791185', 'https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67041009_99_B.jpg?imwidth=2048&imdensity=1&ts=1697727791185']
Gucci1_0.jpg
Gucci1_1.jpg
Gucci1_2.jpg

['https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67057926_91_R.jpg?imwidth=2048&imdensity=1&ts=1710778223851', 'https://shop.mango.com/assets/rcs/pics/static/T6/fotos/S/67057926_91_D8.jp